In [6]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


loading the dataset and creating a random state to make everything repeatable

In [3]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

xss_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/xss_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(xss_dbs))

90


smoke testing run to check everything is working

In [7]:
import tempfile

test_and_train = [
    xss_dbs[0],
]

res_dir = tempfile.mkdtemp()
print("using %s for smoke run" % (res_dir))
# !rm -rf $res_dir/*

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=test_and_train,
    test=test_and_train,
    docker_client=docker_client,
    skip=["train", "evalute_worse", "evaluate_boosted", "evaluate_v0"]
)

INFO:experiments.all:train_and_evaluate with train size 1, test size 1
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /tmp/tmp6ddjrlqd/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes


using /tmp/tmp6ddjrlqd for smoke run


INFO:scripts.evaluate:Evaluating Breeze - breeze.js.samples
/home/pablo/tesis/tsm-pipeline/code/env/lib/python3.10/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/pablo/tesis/tsm-pipeline/code/env/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,NaN,0,0,0,0,0


as a second sanity, doing a run with 10/10 over the first 20 sample

In [8]:
from random import sample
from sklearn.model_selection import train_test_split

sample_20 = sample(xss_dbs, 20)
train, test = train_test_split(sample_20, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

train 10, test 10


In [ ]:
res_dir = tempfile.mkdtemp()
print("using %s for 20 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="xss",
    train=train,
    test=test,
    docker_client=docker_client,
)